In [3]:
import json

import requests

 

#response = requests.get('https://ghoapi.azureedge.net/api/Indicator?$filter=contains(IndicatorName,%20%27Medical%20doctors%27)')
medical_doctors_per_10k = requests.get('https://ghoapi.azureedge.net/api/HWF_0001?$filter=date(TimeDimensionBegin) ge 2017-01-01 and date(TimeDimensionBegin) lt 2018-01-01').json()
#HWF_0001 is the indicator code for medical doctors per 10000, we then filter by date


country_list_code = requests.get('https://ghoapi.azureedge.net/api/DIMENSION/COUNTRY/DimensionValues').json()

In [4]:

import pandas as pd
from pandas import json_normalize

#flattened_data = json_normalize(country_list_code['value'])
country_df = pd.DataFrame(country_list_code['value'])

country_df.head(2)

,Code,Title,ParentDimension,Dimension,ParentCode,ParentTitle
0,AGO,Angola,REGION,COUNTRY,AFR,Africa
1,BDI,Burundi,REGION,COUNTRY,AFR,Africa


In [5]:
doctors_df = pd.DataFrame(medical_doctors_per_10k['value'])[['SpatialDim','NumericValue']]
doctors_df.head(2)

,SpatialDim,NumericValue
0,AGO,2.146
1,ARE,24.356


In [6]:
malaria_estimated_numbers = pd.read_csv('./estimated_numbers.csv')

In [79]:
malaria_estimated_numbers_2017 = malaria_estimated_numbers[malaria_estimated_numbers['Year'].between(2017, 2017)]
malaria_estimated_numbers_2017.head(2)#.sort_values(by=['No. of deaths'])

,Country,Year,No. of cases,No. of deaths,No. of cases_median,No. of cases_min,No. of cases_max,No. of deaths_median,No. of deaths_min,No. of deaths_max,WHO Region
0,Afghanistan,2017,630308[495000-801000],298[110-510],630308,495000.0,801000.0,298,110.0,510.0,Eastern Mediterranean
1,Algeria,2017,0,0,0,NaN,NaN,0,NaN,NaN,Africa


In [8]:
merged_gho_df = country_df.merge(doctors_df, left_on='Code', right_on='SpatialDim')[['Title','ParentTitle','NumericValue']]
merged_gho_df.columns = ['Country', 'Continent','Doctors_per_10k']
merged_gho_df.head(2)

,Country,Continent,Doctors_per_10k
0,Angola,Africa,2.146
1,Burundi,Africa,1.001


In [80]:
gho_plus_malaria_df = merged_gho_df.merge(malaria_estimated_numbers_2017, left_on='Country', right_on='Country')[['Country','Continent','Doctors_per_10k','Year','No. of cases_median','No. of deaths_median']]
gho_plus_malaria_df.columns = ['Country','Continent','Doctors/10k','Year','No of sick People', 'No of dead people']

In [81]:
gho_plus_malaria_df.sort_values(by=['No of dead people'], ascending=False).head(10)


,Country,Continent,Doctors/10k,Year,No of sick People,No of dead people
2,Burkina Faso,Africa,0.847,2017,7907562,27791
46,India,South-East Asia,7.779,2017,9590000,16733
11,Mozambique,Africa,0.761,2017,10025823,14713
6,Uganda,Africa,1.68,2017,8600724,14390
0,Angola,Africa,2.146,2017,4615605,13316
16,Ghana,Africa,0.92,2017,7805045,10917
4,Chad,Africa,0.434,2017,2779489,8729
5,Togo,Africa,0.757,2017,2891868,5341
1,Burundi,Africa,1.001,2017,2113066,5253
8,Senegal,Africa,0.691,2017,1024285,4523


In [ ]:
import mysql.connector
from mysql.connector.constants import ClientFlag
pwd = open("../SQL Token/SQL_pwd", "r")
host = open("../SQL Token/SQL_host", "r")
config = {
    'user': 'Mathieu',
    'password' : pwd.read().strip(),
    'host' : host.read().strip(),
    'client_flags' : [ClientFlag.SSL],
    'ssl_ca': '../SQL Token/server-ca.pem',
    'ssl_cert': '../SQL Token/client-cert.pem',
    'ssl_key': '../SQL Token/client-key.pem'
}

# establish connection
cnxn = mysql.connector.connect(**config)

In [ ]:
# create a database
cursor = cnxn.cursor() # initialize connection cursor
cursor.execute('CREATE DATABASE malaria')
cnxn.close()

In [76]:
# connect to malaria by adding it to config dictionnary
import mysql.connector
from mysql.connector.constants import ClientFlag
pwd = open("../SQL Token/SQL_pwd", "r")
host = open("../SQL Token/SQL_host", "r")
config = {
    'user': 'Mathieu',
    'password' : pwd.read().strip(),
    'host' : host.read().strip(),
    'client_flags' : [ClientFlag.SSL],
    'ssl_ca': '../SQL Token/server-ca.pem',
    'ssl_cert': '../SQL Token/client-cert.pem',
    'ssl_key': '../SQL Token/client-key.pem'
}
config['database'] = 'malaria'
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()
cursor.execute('CREATE TABLE doctors_world ('
                 'Country VARCHAR(255),'
                 'Continent VARCHAR(255),'
                 'Doctors_per_10k FLOAT )')

cnxn.commit() # commit changes to the DB

In [77]:
data = list(merged_gho_df.to_records(index=False))
data = [tuple(x) for x in data] # data contain initialy numpy records and executemany is expecting tuples
# setup the query
query = ("INSERT INTO doctors_world (Country, Continent, Doctors_per_10k) VALUES (%s, %s, %s)")

# execute with every row in dataframe
cursor.executemany(query,data)
cnxn.commit()

In [86]:
cursor.execute("SELECT * FROM doctors_world LIMIT 10")
out = cursor.fetchall()
for row in out:
    print(row)

('Angola', 'Africa', 2.146)
('Burundi', 'Africa', 1.001)
('Burkina Faso', 'Africa', 0.847)
('Zimbabwe', 'Africa', 1.859)
('Chad', 'Africa', 0.434)
('Togo', 'Africa', 0.757)
('Uganda', 'Africa', 1.68)
('South Africa', 'Africa', 7.833)
('Senegal', 'Africa', 0.691)
('Namibia', 'Africa', 5.914)


In [75]:
#cursor.execute('DROP TABLE doctors_world')
#cnxn.commit() # commit changes to the DB

In [66]:
#cursor.execute('TRUNCATE TABLE doctors_world')
#cnxn.commit()

In [84]:
# Create second table
cursor.execute('CREATE TABLE malaria_count ('
                 'Country VARCHAR(255),'
                 'Year SMALLINT,'
                 'Cases BIGINT,'              
                 'Deaths BIGINT )')

cnxn.commit() # commit changes to the DB

In [118]:
malaria_cleaned = malaria_estimated_numbers_2017[['Country','Year','No. of cases_median','No. of deaths_median']]
malaria_cleaned[['Year', 'No. of cases_median', 'No. of deaths_median']] = malaria_cleaned[['Year', 'No. of cases_median', 'No. of deaths_median']].astype(str)

data = list(malaria_cleaned.to_records(index=False))
data = [tuple(x) for x in data] # data contain initialy numpy records and executemany is expecting tuples
# setup the query
query = ("INSERT INTO malaria_count (Country, Year, Cases, Deaths) VALUES (%s, %s, %s, %s)")
#cursor.execute("INSERT INTO malaria_count (Country, Year, Cases, Deaths) VALUES ('Afghanistan', 2017, 630308, 298)")
# execute with every row in dataframe
cursor.executemany(query,data)
cnxn.commit()

/home/osboxes/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [117]:
# Clear Malaria Table - BEWARE
#cursor.execute('TRUNCATE TABLE malaria_count')
#cnxn.commit()

In [1]:
# Connection to database
import mysql.connector
from mysql.connector.constants import ClientFlag
pwd = open("../SQL Token/SQL_pwd", "r")
host = open("../SQL Token/SQL_host", "r")
config = {
    'user': 'Mathieu',
    'password' : pwd.read().strip(),
    'host' : host.read().strip(),
    'client_flags' : [ClientFlag.SSL],
    'ssl_ca': '../SQL Token/server-ca.pem',
    'ssl_cert': '../SQL Token/client-cert.pem',
    'ssl_key': '../SQL Token/client-key.pem'
}
config['database'] = 'malaria'
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

In [131]:
cursor.execute("SELECT * FROM malaria_count LIMIT 2")
out = cursor.fetchall()
for row in out:
    print(row)

('Afghanistan', 2017, 630308, 298)
('Algeria', 2017, 0, 0)


In [130]:
cursor.execute("SELECT * FROM doctors_world LIMIT 2")
out = cursor.fetchall()
for row in out:
    print(row)

('Angola', 'Africa', 2.146)
('Burundi', 'Africa', 1.001)


In [129]:
# Get Table names to prepare groupby
cursor.execute("""SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = N'doctors_world'
""")
out = cursor.fetchall()
for row in out:
    print(row)

('def', 'malaria', 'doctors_world', 'Country', 1, None, 'YES', 'varchar', 255, 765, None, None, None, 'utf8', 'utf8_general_ci', 'varchar(255)', '', '', 'select,insert,update,references', '', '')
('def', 'malaria', 'doctors_world', 'Continent', 2, None, 'YES', 'varchar', 255, 765, None, None, None, 'utf8', 'utf8_general_ci', 'varchar(255)', '', '', 'select,insert,update,references', '', '')
('def', 'malaria', 'doctors_world', 'Doctors_per_10k', 3, None, 'YES', 'float', None, None, 12, None, None, None, None, 'float', '', '', 'select,insert,update,references', '', '')


In [7]:
# Get Data from both table with a JOIN
cursor.execute("""SELECT *
                    FROM doctors_world
                    INNER JOIN malaria_count
                    ON doctors_world.Country = malaria_count.Country""")
out = cursor.fetchall()

In [10]:
import pandas as pd
merged_data = pd.DataFrame(out)#.columns = ['Country', 'Continent','Doctors_per_10k']
merged_data = merged_data.drop([3], axis=1)
merged_data.columns = ['Country', 'Continent','Doctors_per_10k','Year','Malaria Count','Malaria Deaths']
merged_data

,Country,Continent,Doctors_per_10k,Year,Malaria Count,Malaria Deaths
0,Angola,Africa,2.146,2017,4615605,13316
1,Argentina,Americas,39.901,2017,0,0
2,Armenia,Europe,44.023,2017,0,0
3,Bangladesh,South-East Asia,5.433,2017,32924,76
4,Belize,Americas,11.229,2017,7,0
5,Bhutan,South-East Asia,4.024,2017,11,0
6,Bolivia (Plurinational State of),Americas,10.299,2017,6512,2
7,Brazil,Americas,21.652,2017,217928,30
8,Burkina Faso,Africa,0.847,2017,7907562,27791
9,Burundi,Africa,1.001,2017,2113066,5253


In [35]:
total_malaria = merged_data.groupby(by=["Continent"], as_index=True)[["Malaria Count", "Malaria Deaths", "Doctors_per_10k"]].sum()

In [36]:
total_malaria

,Malaria Count,Malaria Deaths,Doctors_per_10k
Continent,,,
Africa,56421684,115187,40.040
Americas,846929,500,264.270
Eastern Mediterranean,2474427,4690,116.885
Europe,0,0,123.812
South-East Asia,11287365,19719,100.432
Western Pacific,36401,64,53.136


In [39]:
total_malaria['% Malaria Deaths'] = total_malaria['Malaria Deaths'] / total_malaria['Malaria Deaths'].sum() * 100
total_malaria['% Doctors'] = total_malaria['Doctors_per_10k'] / total_malaria['Doctors_per_10k'].sum() * 100
total_malaria

,Malaria Count,Malaria Deaths,Doctors_per_10k,% Malaria Deaths,% Doctors
Continent,,,,,
Africa,56421684,115187,40.040,82.182506,5.731668
Americas,846929,500,264.270,0.356735,37.829868
Eastern Mediterranean,2474427,4690,116.885,3.346176,16.731919
Europe,0,0,123.812,0.000000,17.723509
South-East Asia,11287365,19719,100.432,14.068921,14.376695
Western Pacific,36401,64,53.136,0.045662,7.606341
